In [1]:
import pandas as pd 
import numpy  as np 
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from statsmodels.tsa.seasonal import STL
from sklearn import metrics
import hydroeval as he 
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from hyperopt import hp, tpe, fmin, Trials,space_eval
import lightgbm as lgb

In [2]:
def create_dataset2(data,days_for_train,step):
    dataset_x,dataset_y = [],[]
    l=len(data)-days_for_train-(step-1)
    dataset_y=np.zeros((l,step))
    for i in range(l):
        _x = data[i:i+days_for_train]
        dataset_x.append(_x)
        a=data.iloc[i+days_for_train:i+days_for_train+step].values.reshape(-1,)
        dataset_y[i]=a
    return (np.array(dataset_x).reshape(len(dataset_x),days_for_train),
            dataset_y[:,-1])
"""Enter data DataFrame, seq length and predicted step size, training and validation size"""
def train_valid_data(data2,seq,step,valid_size=None):
    seq_len = seq
    data_len = len(data2)
    step=step
    days=data_len-seq_len-step+1
    data2=np.array(data2)
    x = np.zeros((days,seq_len,4))
    y = np.zeros((days,seq_len,1))
    for i in range(0,days):
        x[i] = data2[i:i+seq_len]
        y[i] = data2[i+step:i+seq_len+step][:,0].reshape(-1,1)

    x=x.reshape(x.shape[0],x.shape[1]*x.shape[2])
    y=y.reshape(y.shape[0],y.shape[1])[:,-1]
    
    return   x,y

In [3]:
def stl(data):
    res = STL(data).fit()
    
    data2=pd.concat([data,res.seasonal,res.trend,res.resid],axis=1)

    data2 = np.hstack((np.array(data2).reshape(-1,)))
#     return np.array(data2).reshape((data2.shape[0]*data2.shape[1],res.seasonal[-3:],res.trend[-3:],res.resid[-3:]))
    return np.array(data2)
def stl_data(data,days_for_train,step):
    dataset_x,dataset_y = [],[]
    l=len(data)-days_for_train-(step-1)
    dataset_y=np.zeros((l,step))
    for i in range(l):
        _x = stl(data[i:i+days_for_train])
        dataset_x.append(_x)
        a=data.iloc[i+days_for_train:i+days_for_train+step].values.reshape(-1,)
        dataset_y[i]=a
    return (np.array(dataset_x).reshape(len(dataset_x),days_for_train*4),
            dataset_y[:,-1])

In [7]:

# Hyperparameter search space
hyperparameter_space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'max_depth': hp.choice('max_depth', np.arange(3, 15, dtype=int)),
    'num_leaves': hp.choice('num_leaves', np.arange(10, 100, dtype=int)),
    'min_data_in_leaf': hp.choice('min_data_in_leaf', np.arange(1, 50, dtype=int)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}
def objective(params, train_features, train_labels):
    
    model = LGBMRegressor(**params,random_state=42, n_jobs=-1,force_col_wise=True)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    # Calculate 5 fold cross validation MSE
    cv_scores = -cross_val_score(model, train_features, train_labels, cv=kf, scoring='neg_mean_squared_error')
    mean_score = cv_scores.mean()
    
    return mean_score

In [ ]:
def lgb(train_features, train_labels, test_features, test_labels, max_evals=100):
    trials = Trials()
    # hyparameter optimization
    best_params = fmin(fn=lambda p: objective(p, train_features, train_labels),
                       space=hyperparameter_space,
                       algo=tpe.suggest,
                       max_evals=max_evals,
                       trials=trials)


    best_hyperparams = space_eval(hyperparameter_space, best_params)
    
    # training model
    model = LGBMRegressor(**best_hyperparams, random_state=42, n_jobs=-1,force_col_wise=True)
    model.fit(train_features, train_labels)
    
    # prediction
    pre_train = model.predict(train_features)
    pre_test = model.predict(test_features)
    
    #  RMSE and NSE
    rmse = np.sqrt(metrics.mean_squared_error(pre_test, test_labels))
    nse_score = he.evaluator(he.nse,pre_test, test_labels)

    
    return np.round(nse_score,3),np.round(rmse,3),pre_test,test_labels,best_hyperparams

In [9]:
def all_model_stl(train,test,lagtime,leadingtime,s=None):

    train_features,train_labels=train_valid_data(train,lagtime,leadingtime)

    if s==3:
        test_features,test_labels=stl_data(test,lagtime,leadingtime)
    else:
        test_features,test_labels=train_valid_data(test,lagtime,leadingtime)
        
    lgb_relust=lgb(train_features,train_labels,test_features,test_labels)

    l=(lgb_relust)


    return l

def all_model(train,test,lagtime,leadingtime):

    train_features,train_labels=create_dataset2(train,lagtime,leadingtime)
    test_features,test_labels=create_dataset2(test,lagtime,leadingtime)

    lgb_relust=lgb(train_features,train_labels,test_features,test_labels)

    l=(lgb_relust)
    return l 

def stl_model(train,test,lagtime,leadingtime,train_base,test_base,s=None):
    l=all_model_stl(train,test,lagtime,leadingtime,s)
    l1=all_model(train_base,test_base,lagtime,leadingtime)
    return l ,l1 

In [ ]:
df = pd.read_excel('flow2021.xlsx').iloc[:,[0,1]]#Read the data
data=df.set_index('date')#Set the date as an index
data = data.resample('D').mean().ffill()
test_1=data.iloc[int(0.6*len(data)):,:]#STL-S2 testing data
train=data.iloc[:int(0.6*len(data)),:]

res = STL(train).fit()
train=pd.concat([train,res.seasonal,res.trend,res.resid],axis=1)
train
test=data.iloc[int(0.6*len(data)):,:]
res_test = STL(test).fit()
test=pd.concat([test,res_test.seasonal,res_test.trend,res_test.resid],axis=1)
test#STL-S1  testing data
train_base=df.iloc[:int(0.6*len(data)),1]
test_base=df.iloc[int(0.6*len(data)):,1]

s2_NSE=[]
s2_RMSE=[]

s1_NSE=[]
s1_RMSE=[]

base_NSE=[]
base_RMSE=[]

s1_pre=[]
s2_pre=[]
base_pre=[]
obs=[]

sliding_window=12

for i in range(1,8):
    ltime=i
    result,result_base=stl_model(train,test,sliding_window,ltime,train_base,test_base)
    result3=stl_model(train,test_1,sliding_window,ltime,train_base,test_base,3)[0]
    s1_NSE.append(result[0])
    s1_RMSE.append(result[1])
    s2_NSE.append(result3[0])
    s2_RMSE.append(result3[1])
    
    base_NSE.append(result_base[0])
    base_RMSE.append(result_base[1])
    
    s1_pre.append(result[2])
    s2_pre.append(result3[2])
    obs.append(result[3])
    base_pre.append(result_base[2])
    
pd1=np.vstack([s1_NSE,s1_RMSE,s2_NSE,s2_RMSE,base_NSE,base_RMSE,]).T
pd1=pd.DataFrame(pd1,columns=['S1_NSE','S1_RMSE','S2_NSE','S2_RMSE','base_NSE','base_RMSE',])
pd1.to_excel('S1+S2+base.xlsx')

pd.DataFrame(s1_pre).T.to_excel('S1_pre.xlsx')
pd.DataFrame(s2_pre).T.to_excel('S2_pre.xlsx')
pd.DataFrame(base_pre).T.to_excel('base_pre.xlsx')
pd.DataFrame(obs).T.to_excel('obs2.xlsx')